In [2]:
from factCheck import factCheckSingleClaim

# models = ['llama3', 'llama3.2']
# models = ['llama3.2']
claimCount = 15

In [3]:
from sklearn.metrics import accuracy_score, f1_score

# Evaluation function
def evaluate_model(predictions, y):
    for idx in range(len(predictions)):
        if y[idx] in predictions[idx]:
            predictions[idx] = y[idx];
        else:
            predictions[idx] = predictions[idx][0];
    
    # Ensure predictions and dataset labels are aligned
    accuracy = accuracy_score(y, predictions)
    f1 = f1_score(y, predictions, average='weighted')
    return accuracy, f1

In [6]:
from benchmarking.AVERITEC.feverEval import get_fever_sample

dfFever = get_fever_sample(claimCount, './benchmarking/AVERITEC/data_train.json', 42)

In [8]:
feverMapping = {
    "true": ["Supported"],
    "mostly true": ["Supported", "Conflicting Evidence/Cherrypicking"],
    "mostly false": ["Refuted", "Conflicting Evidence/Cherrypicking"],
    "false": ["Refuted"],
    "not enough evidence": ["Not Enough Evidence", "Conflicting Evidence/Cherrypicking"]
}
origPred = []
feverPred = []
for row in dfFever.iterrows():
      
    pred = await factCheckSingleClaim(row[1]['claim'])
    origPred.append(pred['label'].lower())
    pred['label'] = feverMapping[pred['label'].lower()]
    feverPred.append(pred['label'])

https://www.factcheck.org/search/#gsc.tab=0&gsc.q=Cyril%20Ramaphosa%3A%20South%20Africa%20has%20launched%20a%20Youth%20Employment%20Service%2C%20which%20places%20unemployed%20youth%20in%20paid%20internships%20in%20companies&gsc.sort=
FactCheck Articles: []
{'status': 'ok', 'totalResults': 0, 'articles': []}
News Articles: []


/home/root/work/factCheck.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=ollama_model, prompt=prompt)
/home/root/work/factCheck.py:143: LangChainDeprecationWarning: The method `Chain.arun` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~ainvoke` instead.
  response = await chain.arun({"claim": claim})




RESULT: 


Based on my analysis, here are the findings:

1. Classification: Mostly True
2. Background Information:
In 2020, South Africa launched the National Youth Employment Service (NYES) as part of its efforts to tackle unemployment among young people. The program aims to provide opportunities for unemployed youth to gain work experience and skills through paid internships in various companies.
3. Key Facts and Evidence:

* The NYES was announced by President Cyril Ramaphosa in October 2020 as a key component of the government's job creation strategy.
* The program is designed to provide an estimated 750,000 young people with employment opportunities over the next three years.
* The service will offer internships in various sectors, including agriculture, mining, and manufacturing.
* Companies participating in the program will be required to hire at least 10% of their workforce from the NYES.

Relevant sources:
* South African Government's website (www.gov.za)
* News articles fro

In [10]:
feverAcc = evaluate_model(feverPred, dfFever['label'].values)
feverAcc

(0.4666666666666667, 0.6102564102564102)